In [45]:
import torch
from torch import nn
from d2l import torch as d2l

In [46]:
def corr2d_multi_in(X, K):
    # 遍历 X 和 K 的 dim0（通道维度），再相加
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [47]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [48]:
def corr2d_multi_in_out(X, K):
    # 迭代 K 的 dim0 与 X 进行互相关运算
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [49]:
# 若干个张量在 dim 维度上连接，生成一个扩维的张量
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [50]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [51]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [52]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

In [53]:
# 练习6
def corr2d_matrix(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    K = K.reshape((1, -1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = K @ X[i: i + h, j: j + w].reshape((-1, 1))
    return Y

def corr2d_multi_in_matrix(X, K):
    return sum(corr2d_matrix(x, k) for x, k in zip(X, K))

def corr2d_multi_in_out_matrix(X, K):
    return torch.stack([corr2d_multi_in_matrix(X, k) for k in K], 0)

In [54]:
X_matrix = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K_matrix = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in_matrix(X_matrix, K_matrix)
# 若干个张量在 dim 维度上连接，生成一个扩维的张量
K_matrix = torch.stack((K_matrix, K_matrix + 1, K_matrix + 2), 0)
K_matrix.shape

torch.Size([3, 2, 2, 2])

In [55]:
corr2d_multi_in_out_matrix(X_matrix, K_matrix)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])